In [1]:
import pandas as pd
import json
from pymongo import MongoClient
from pprint import pprint

Importo db_STD.json a cui sono stati aggiunte nuove colonne le seguenti colonne: 

     • 'avg_score', rappresenta il punteggio di gradimento ottenuto calcolato come la media di score2 dentro la colonna 'tweet' 
        diviso il numero di tweets ottenuti dallo screaping;
     • 'IMBD_Rating_std', è il punteggio 'IMDB_Rating' standardizzato;
     • 'avg_score_std', è il punteggio 'avg_score' standardizzato;
     • 'diff_std_rating', differenza fra i punteggi IMBD_Rating_std e avg_score_std
    
Il nuovo database viene caricato in Mongo Express, in modo da non dover lavorare in locale, e tramite il linguaggio MongoDB si effettuano delle interrogazioni utiliti per raggiungere gli obiettivi prefissati.

In [5]:
with open ('db_STD.json') as f:
    data1 = json.load(f)

In [6]:
data1_df = pd.DataFrame(data1)

In [8]:
data1_df.head(2)

,_id,show_id,type,title,director,cast,country,rating,duration,Certificate,Runtime_of_Episodes,Genre,IMDB_Rating,No_of_Votes,tweet,avg_score,IMDB_Rating_std,avg_score_std,diff_std_rating
0,{'$oid': '60deeaa603da8f55eb4792e5'},s1026,TV Show,BoJack Horseman,None,"Will Arnett, Aaron Paul, Amy Sedaris, Alison B...",United States,TV-MA,6 Seasons,18+,25 min,"Animation, Comedy, Drama",8.7,113345,"[{'_id': '60dae6af60d63d1fa250cb25', 'text': '...",0.04625,1.151325,0.110309,1.041016
1,{'$oid': '60deeaa603da8f55eb4792e6'},s1052,TV Show,Borgen,None,"Sidse Babett Knudsen, Birgitte Hjort Sørensen,...",Denmark,TV-MA,3 Seasons,16,58 min,Drama,8.5,19655,"[{'_id': '60dae6af60d63d1fa2508b1a', 'text': '...",0.15100,0.928795,0.631056,0.297739


In [9]:
data1_df.drop(['_id'], axis = 1, inplace = True)

In [10]:
data1_df.head(2)

,show_id,type,title,director,cast,country,rating,duration,Certificate,Runtime_of_Episodes,Genre,IMDB_Rating,No_of_Votes,tweet,avg_score,IMDB_Rating_std,avg_score_std,diff_std_rating
0,s1026,TV Show,BoJack Horseman,None,"Will Arnett, Aaron Paul, Amy Sedaris, Alison B...",United States,TV-MA,6 Seasons,18+,25 min,"Animation, Comedy, Drama",8.7,113345,"[{'_id': '60dae6af60d63d1fa250cb25', 'text': '...",0.04625,1.151325,0.110309,1.041016
1,s1052,TV Show,Borgen,None,"Sidse Babett Knudsen, Birgitte Hjort Sørensen,...",Denmark,TV-MA,3 Seasons,16,58 min,Drama,8.5,19655,"[{'_id': '60dae6af60d63d1fa2508b1a', 'text': '...",0.15100,0.928795,0.631056,0.297739


In [20]:
data1 = data1_df.to_dict(orient = 'records')

In [21]:
client.Query.Df_query_avg_norm_intero.insert_many(data1)

In [22]:
db = client.Query
database = db.Df_query_avg_norm_intero

### Raggruppo per country e calcolo lo score medio dell'avg dello score2 normalizzato

In [57]:
results = database.aggregate([
   {'$group':{
          '_id': '$country',
          'valor_medio_sentiment': {'$avg': '$avg_score_std'}
        }
    },
    { '$project': { 'valor_medio_sentiment': { '$round': [ "$valor_medio_sentiment", 3 ] } } },

])
[result for result in results]

[{'_id': 'Canada, United Kingdom', 'valor_medio_sentiment': -2.317},
 {'_id': 'United Kingdom, United States', 'valor_medio_sentiment': -0.246},
 {'_id': 'Italy', 'valor_medio_sentiment': -0.018},
 {'_id': 'Ireland, United Kingdom, United States',
  'valor_medio_sentiment': -2.031},
 {'_id': 'United States, Colombia, Mexico', 'valor_medio_sentiment': 0.212},
 {'_id': 'Germany', 'valor_medio_sentiment': -0.209},
 {'_id': 'United Kingdom, Canada, United States',
  'valor_medio_sentiment': 1.282},
 {'_id': 'France, Canada', 'valor_medio_sentiment': -0.12},
 {'_id': None, 'valor_medio_sentiment': 0.255},
 {'_id': 'United Kingdom, Spain', 'valor_medio_sentiment': 0.527},
 {'_id': 'United States, Mexico, Spain, Malta',
  'valor_medio_sentiment': -0.685},
 {'_id': 'France', 'valor_medio_sentiment': 0.136},
 {'_id': 'Denmark', 'valor_medio_sentiment': 0.065},
 {'_id': 'Turkey', 'valor_medio_sentiment': 1.9},
 {'_id': 'India', 'valor_medio_sentiment': -0.511},
 {'_id': 'Denmark, United States',

### Raggruppo per stagioni e calcolo lo score medio normalizzato

In [45]:
results = database.aggregate([
   {'$group': {
          '_id': '$duration',
          'valor_medio_sentiment': {'$avg': '$avg_score_std'}
        }
    },
    { '$sort': { '_id' : 1 } },
    { '$project': { 'valor_medio_sentiment': { '$round': [ "$valor_medio_sentiment", 3 ] } } },
])
[result for result in results]

[{'_id': '1 Season', 'valor_medio_sentiment': -0.13},
 {'_id': '10 Seasons', 'valor_medio_sentiment': 1.101},
 {'_id': '11 Seasons', 'valor_medio_sentiment': 0.644},
 {'_id': '15 Seasons', 'valor_medio_sentiment': 0.198},
 {'_id': '2 Seasons', 'valor_medio_sentiment': 0.293},
 {'_id': '3 Seasons', 'valor_medio_sentiment': 0.141},
 {'_id': '4 Seasons', 'valor_medio_sentiment': -0.225},
 {'_id': '5 Seasons', 'valor_medio_sentiment': -0.336},
 {'_id': '6 Seasons', 'valor_medio_sentiment': 0.294},
 {'_id': '7 Seasons', 'valor_medio_sentiment': -0.423},
 {'_id': '8 Seasons', 'valor_medio_sentiment': 0.146},
 {'_id': '9 Seasons', 'valor_medio_sentiment': -1.032}]

### Raggruppo per country e calcolo lo score medio della sentiment e di IMDB normalizzati

In [56]:
results = database.aggregate([
    {'$group':{
          '_id': '$country',
          'valor_medio_sentiment': {'$avg': '$avg_score_std'},
          'valor_medio_IMDB': {'$avg': '$IMDB_Rating_std'}
        }
    },
    { '$project': { 'valor_medio_sentiment': { '$round': [ "$valor_medio_sentiment", 3 ] },
                   'valor_medio_IMDB': { '$round': [ "$valor_medio_IMDB", 3 ] }, }
    },
    
])
[result for result in results]

[{'_id': 'Canada, United Kingdom',
  'valor_medio_sentiment': -2.317,
  'valor_medio_IMDB': 0.706},
 {'_id': 'United Kingdom, United States',
  'valor_medio_sentiment': -0.246,
  'valor_medio_IMDB': 1.596},
 {'_id': 'Italy', 'valor_medio_sentiment': -0.018, 'valor_medio_IMDB': -0.963},
 {'_id': 'Ireland, United Kingdom, United States',
  'valor_medio_sentiment': -2.031,
  'valor_medio_IMDB': 0.595},
 {'_id': 'United States, Colombia, Mexico',
  'valor_medio_sentiment': 0.212,
  'valor_medio_IMDB': 1.263},
 {'_id': 'Germany',
  'valor_medio_sentiment': -0.209,
  'valor_medio_IMDB': -0.518},
 {'_id': 'United Kingdom, Canada, United States',
  'valor_medio_sentiment': 1.282,
  'valor_medio_IMDB': 1.04},
 {'_id': 'France, Canada',
  'valor_medio_sentiment': -0.12,
  'valor_medio_IMDB': 0.261},
 {'_id': None, 'valor_medio_sentiment': 0.255, 'valor_medio_IMDB': -1.334},
 {'_id': 'United Kingdom, Spain',
  'valor_medio_sentiment': 0.527,
  'valor_medio_IMDB': -1.185},
 {'_id': 'United States,